In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


In [2]:
import sys
sys.path.insert(0, '../../')   # для обращения к родительской директории

#from notebook.utils import create_sliding_window, restore_labels

In [30]:
# должны быть четные числа:
window_size = 30
shift=4

def create_sliding_window(X, y=None, window_size=40, shift=1):
    """
    Создает набор данных с окнами, скользящими по временному ряду.

    Аргументы:
    X - временной ряд, y - метки
    window_size - размер окна
    shift - сдвиг окна

    Возвращает кортеж: временной массив с окнами,
    массив с метками для каждого окна
    """

    dataset, labels = [], []

    for i in range(0, len(X) - window_size, shift):
        X_window = X[i:(i + window_size)]
        dataset.append(X_window)

        if y is not None:
            y_window = y[i:(i + window_size)]
            max_label = np.argmax(np.bincount(y_window, minlength=3))
            labels.append(max_label)

    return np.array(dataset), np.array(labels) if y is not None else None


In [4]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [5]:
#ЭКСГАУСТЕР №5

X_columns = [
       'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 1',
       'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 2',
       'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 3',
       'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.',
       'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 4',
       'ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.']

In [6]:
X = pd.read_parquet('../../data/processed/ex5_M3_X_all_for_y_5.parquet', columns=X_columns)
X_test = pd.read_parquet('../../data/X_test.parquet', columns=X_columns)
y = pd.read_parquet('../../data/processed/ex5_M3_y_5.parquet')

In [7]:
X.head(3)

,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 1,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 2,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3. ПРОДОЛЬНАЯ.,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4,ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
DT,,,,,,
2019-01-16 13:21:00,1.600000,0.760000,0.670000,1.63000,3.45,0.740000
2019-01-16 13:21:10,1.572917,0.751736,0.642361,1.55816,3.47,0.734375
2019-01-16 13:21:20,1.553819,0.735918,0.638503,1.58179,2.90,0.748457


In [8]:
y = y.squeeze()  # Series
y.head()

DT
2019-01-16 13:21:00    0.0
2019-01-16 13:21:10    0.0
2019-01-16 13:21:20    0.0
2019-01-16 13:21:30    0.0
2019-01-16 13:21:40    0.0
Name: Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №4, dtype: float64

In [9]:
print(X.shape)
print(X_test.shape)
print(y.shape)

(3027868, 6)
(4008961, 6)
(3027868,)


In [10]:
# Масштабирование
scaler = MinMaxScaler()

#X_norm = scaler.fit_transform(np.array(X).reshape(-1, 1)).flatten()
X_norm = scaler.fit_transform(X)
X_test_norm = scaler.transform(X_test)

X_train = X_norm[:2500000]
y_train = y[0:2500000]

X_val = X_norm[2500000:]
y_val = y[2500000:]

print(y_train.value_counts())
print(y_val.value_counts())

Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №4
0.0    2266719
2.0     233281
Name: count, dtype: int64
Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №4
2.0    311509
0.0    216359
Name: count, dtype: int64


In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(2500000, 6)
(2500000,)
(527868, 6)
(527868,)


In [12]:

# Создаем набор данных с окнами
X_train_window, y_train_window = create_sliding_window(X_train, y_train, window_size, shift=shift)
X_val_window, y_val_window = create_sliding_window(X_val, y_val, window_size, shift=shift)
X_test_window, _ = create_sliding_window(X_test_norm, window_size=window_size, shift=shift)

print(X_train_window.shape)
print(y_train_window.shape)

# Кодирование
y_train_encoded = to_categorical(y_train_window, num_classes=3)
y_val_encoded = to_categorical(y_val_window, num_classes=3)

# Зачем
# Преобразуем в формат TensorFlow
X_train_window = tf.convert_to_tensor(X_train_window)
y_train_window = tf.convert_to_tensor(y_train_encoded)
X_val_window = tf.convert_to_tensor(X_val_window)
y_val_window = tf.convert_to_tensor(y_val_encoded)
X_test_window = tf.convert_to_tensor(X_test_window)

print('После преобразования в формат TensorFlow:')
print(X_train_window.shape)
print(y_train_window.shape)
print(X_val_window.shape)
print(y_val_window.shape)
print(X_test_window.shape)


(624993, 30, 6)
(624993,)
После преобразования в формат TensorFlow:
(624993, 30, 6)
(624993, 3)
(131960, 30, 6)
(131960, 3)
(1002233, 30, 6)


In [13]:
input_dim = X_train_window.shape[2]

In [14]:
# Модель
model = tf.keras.Sequential([
    layers.Conv1D(32, kernel_size=5, activation='relu', input_shape=(window_size, input_dim)),
    layers.Conv1D(32, kernel_size=5, activation='relu'),
    layers.MaxPooling1D(pool_size=3),
    layers.LSTM(64),    # return_sequences, return_state
    layers.Dropout(0.2),
    layers.Dense(64),
    layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 26, 32)            992       
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 32)            5152      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 7, 32)            0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [16]:
# Обучение на объединенной выборке

history = model.fit(X_train_window, y_train_window,
                    epochs=50,
                    shuffle=False,
                    validation_data=(X_val_window, y_val_window),
                    callbacks=[early_stop])

Epoch 1/50
19532/19532 [==============================] - 134s 7ms/step - loss: 0.0067 - accuracy: 0.9992 - val_loss: 13.4315 - val_accuracy: 0.4099
Epoch 2/50
19532/19532 [==============================] - 130s 7ms/step - loss: 0.0089 - accuracy: 0.9992 - val_loss: 14.2816 - val_accuracy: 0.4099
Epoch 3/50
19532/19532 [==============================] - 129s 7ms/step - loss: 0.0083 - accuracy: 0.9993 - val_loss: 13.7536 - val_accuracy: 0.4099
Epoch 4/50
19532/19532 [==============================] - 130s 7ms/step - loss: 0.0079 - accuracy: 0.9992 - val_loss: 13.2445 - val_accuracy: 0.4099
Epoch 5/50
19532/19532 [==============================] - 130s 7ms/step - loss: 0.0098 - accuracy: 0.9991 - val_loss: 15.5661 - val_accuracy: 0.4099
Epoch 6/50
19532/19532 [==============================] - 132s 7ms/step - loss: 0.0123 - accuracy: 0.9990 - val_loss: 15.9694 - val_accuracy: 0.4099
Epoch 7/50
19532/19532 [==============================] - 136s 7ms/step - loss: 0.0167 - accuracy: 0.9987 

In [20]:
#tf.keras.models.save_model(model, '../../models/ex5_M3_X_10-15_for_y_5')

Прогноз для файла X_test

In [17]:
# X_test содержит интервалы с NaN
pred_window = model.predict(X_test_window)
print(pred_window.shape)

# округление вероятности + декодирование (по индексам столбцов)
pred_window = np.argmax(pred_window, axis=1)  
print(pred_window.shape)

31320/31320 [==============================] - 75s 2ms/step
(1002233, 3)
(1002233,)


In [18]:
np.unique(pred_window)

array([0], dtype=int64)

In [17]:
def restore_labels(y_window, window_size, shift=1):
    """
    Восстанавливает массив меток исходной размерности из массива с окнами.
    Значению присваивается метка из первого окна последовательности.
    
    """
    labels = []
    
    labels.extend([y_window[0]] * (window_size - shift))
    
    for i in range(len(y_window)):
        labels.extend([y_window[i]] * shift)

    return np.array(labels)

In [20]:
pred = restore_labels(pred_window, window_size=window_size, shift=shift)
pred.shape   # X_test.shape = (4008961, 6)

(4008958,)

In [22]:
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [31]:
sub_2 = pd.read_parquet('../../results/submission_2.parquet')
print(sub_2.shape)
sub_2.head()

(4008961, 175)


,Y_ЭКСГАУСТЕР А/М №9_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9,Y_ЭКСГАУСТЕР А/М №9_МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_КЛ1 ТР№9 ДО ЭД ЭКСГАУСТЕРА №9,Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_КОРПУС ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_МАСЛООХЛАДИТЕЛЬ М-05-1 ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_ЭЛ/ДВИГАТЕЛЬ ГАЗ. ЗАДВИЖКИ ЭКСГ. №9,Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №9,...,Y_ЭКСГАУСТЕР А/М №6_ЭКСГАУСТЕР А/М №6,Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №6,Y_ЭКСГАУСТЕР А/М №6_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ6 ВУ1,Y_ЭКСГАУСТЕР А/М №6_ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №6,Y_ЭКСГАУСТЕР А/М №6_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (ПУСК.) ЭКСГ.№6,Y_ЭКСГАУСТЕР А/М №6_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№6 Т.3,Y_ЭКСГАУСТЕР А/М №6_МАСЛОПРОВОДЫ ЭКСГ №6,Y_ЭКСГАУСТЕР А/М №6_КЛ1 ТР№6 ДО ЭД ЭКСГАУСТЕРА №6,Y_ЭКСГАУСТЕР А/М №6_КЛ2 ТР№6 ДО ЭД ЭКСГАУСТЕРА №6,Y_ЭКСГАУСТЕР А/М №6_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№6 Т.2
index,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 00:00:10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 00:00:20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 00:00:30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-01-01 00:00:40,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1]:
# Предсказанные данные подставляем в файл и сохраняем
col_index = sub_2.columns.get_loc(y.name)
sub_2.iloc[:len(pred), col_index] = pred

sub_2['Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №5'].head()

NameError: name 'sub_2' is not defined

In [33]:
sub_2.to_parquet('../../results/submission_2.parquet')